In [20]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
from datetime import timedelta, date
import sys
import pandas as pd
import time


def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
add_link = "bulli/2033"
race_date = "11-11-1111"


col = ['date','race_name','race_number','race_place', 'rug', 'dog_name', 'dog_trainer', 'time', 'mgn', 'split', 'inRun', 'wgt', 'sire', 'dam', 'sp']
all_data = pd.DataFrame([],columns=col)

In [13]:
def race_scrape():
    base_link = "https://thegreyhoundrecorder.com.au/results/"

    race_name = add_link
    link = base_link+add_link
    print(link)

    raw_html = simple_get(link) #Connection complete
    print(len(raw_html))

    html = BeautifulSoup(raw_html, 'html.parser')

    html_file_name='race.html'
    orig_stdout = sys.stdout
    f = open(html_file_name, 'w')
    sys.stdout = f

    print(html.prettify())

    sys.stdout = orig_stdout
    f.close()





    raceContent = html.findAll('div',{"class": "resultsDesktopContent tabs"})[0]

    raceTable = raceContent.findAll('table')
    no_of_races = int(len(raceTable)/2)
    #print(no_of_races)


    i=0 # race iterator
    while(i<no_of_races):

        # ##printing the unique race header

        raceHeader = raceTable[2*i]
        raceHeader = raceHeader.findAll('td')
        #print(len(raceHeader))

        raceNumber = raceHeader[0].decode_contents()
        raceSubName = raceHeader[1].decode_contents()
        raceLength = raceHeader[2].decode_contents()
        raceHeaderCategory = raceHeader[3].decode_contents()

        raceBets = raceHeader[4].decode_contents()
        raceBets = raceBets.replace(' ','')
        raceBets = raceBets[:(raceBets.rfind('-')+1)]+raceBets[raceBets.rfind('$'):]

        raceSplits = raceHeader[5].decode_contents()
        #print(raceNumber+raceSubName+raceLength+raceHeaderCategory+raceBets+raceSplits)

        raceBody = raceTable[2*i+1]
        raceBody = raceBody.find('tbody')
        rows = raceBody.findAll('tr')
        no_of_rows = len(rows)

        #printing the rest of the table

        j = 0 #row iterator
        while(j<no_of_rows):
            current_row = rows[j].findAll('td')

            #print(current_row)

            race_place = current_row[0].decode_contents()
            rug = current_row[1].decode_contents()
            dog_name = current_row[2].find('a')['href'][12:]
            dog_trainer = current_row[3].find('a')['href'][10:]
            time = current_row[4].decode_contents()
            mgn = current_row[5].decode_contents()
            split = current_row[6].decode_contents()
            inRun = current_row[7].decode_contents()
            wgt = current_row[8].decode_contents()
            sire = current_row[9].find('a')['href'][12:]
            dam = current_row[10].find('a')['href'][12:]
            sp = current_row[11].find('p').decode_contents()[2:]

            #print(race_place+' '+rug+' '+dog_name+' '+dog_trainer+' '+time+' '+mgn+' '+split+' '+inRun+' '+wgt+' '+sire+' '+dam+' '+sp)

            df2 = pd.DataFrame([[race_date, race_name, (i+1), race_place, rug, dog_name, dog_trainer, time, mgn, split, inRun, wgt, sire, dam, sp]],columns=col)
            global all_data
            all_data = all_data.append(df2)
            j=j+1
        i=i+1
    

In [14]:
print(all_data)

Empty DataFrame
Columns: [date, race_name, race_number, race_place, rug, dog_name, dog_trainer, time, mgn, split, inRun, wgt, sire, dam, sp]
Index: []


<class 'pandas.core.frame.DataFrame'>


In [16]:
dateFile = pd.read_csv('date.csv')
print(dateFile)

                       0
0             2016-01-01
1             bulli/2033
2         cambridge/2035
3            darwin/2032
4             dubbo/2031
5           geelong/2025
6           gosford/2027
7           ipswich/2029
8          mandurah/2034
9       strathalbyn/2030
10        traralgon/2026
11            wagga/2028
12            2016-01-02
13        addington/2020
14        bundaberg/2022
15          ipswich/2023
16         mandurah/2016
17         richmond/2015
18         tamworth/2017
19      the-gardens/2014
20      the-meadows/2012
21      warrnambool/2013
22         wauchope/2021
23   wentworth-park/2018
24            young/2019
25            2016-01-03
26      albion-park/2011
27           cairns/2009
28           gawler/2010
29      healesville/2005
..                   ...
344         hatrick/1685
345         ipswich/1690
346        mandurah/1688
347            sale/1683
348     strathalbyn/1684
349     the-gardens/1692
350           2016-01-30
351        armidale/1674


In [17]:
print(dateFile.iloc[10,0])

traralgon/2026


In [21]:



t = time.process_time()




len_of_date = len(dateFile)
k=0
while(k<len_of_date):
    print(dateFile.iloc[k,0])
    if(dateFile.iloc[k,0].find('/')==-1):
        race_date = dateFile.iloc[k,0]
    else:
        add_link = dateFile.iloc[k,0]
        race_scrape()
    k=k+1
    
    
elapsed_time = time.process_time() - t

2016-01-01
bulli/2033
https://thegreyhoundrecorder.com.au/results/bulli/2033
291626
cambridge/2035
https://thegreyhoundrecorder.com.au/results/cambridge/2035
327685
darwin/2032
https://thegreyhoundrecorder.com.au/results/darwin/2032
230821
dubbo/2031
https://thegreyhoundrecorder.com.au/results/dubbo/2031
306420
geelong/2025
https://thegreyhoundrecorder.com.au/results/geelong/2025
335506


TypeError: 'NoneType' object is not subscriptable

In [10]:
print(type(all_data))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
all_data.to_csv('out.csv',index=False)
print(type(all_data))